## Pickle all the things

we scan all the images, and run them through the pipeline to create tensors. 
We than save that as a .pkl file for later usage



In [ ]:
!pip install -q optimum[onnxruntime]==1.23.3 transformers==4.46.3 torch==2.2.2+cu121
import time
import requests

In [ ]:
from optimum.onnxruntime import ORTModelForImageClassification
from transformers import AutoFeatureExtractor

# Load and export the model to ONNX
model = ORTModelForImageClassification.from_pretrained("rh-ai-bu/wildfire01", export=True)

# Load the feature extractor
feature_extractor = AutoFeatureExtractor.from_pretrained("rh-ai-bu/wildfire01")

# Create an ONNX-based pipeline
from optimum.pipelines import pipeline
onnx_pipeline = pipeline("image-classification", model=model, feature_extractor=feature_extractor, accelerator="ort")

In [ ]:
onnx_pipeline.save_pretrained("onnx_pipeline")

In [ ]:
import os
import glob
import pickle
import time
import requests

def find_image_files(path):
    # List of image file extensions
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.gif']

    # Create a list to hold full paths of image files
    image_files = []
    files_to_process = 1  # Initialize a counter for processed files

    # Iterate over each file extension
    for ext in image_extensions:
        # Use glob to find all files with the current extension in the given path and its subdirectories
        for file in glob.iglob(os.path.join(path, '**', ext), recursive=True):
            # Append the full path to the list
            image_files.append(file)
            if files_to_process >= 50:  # Check if we've processed 50 files
                return image_files  # If so, exit the loop
            files_to_process += 1  # Increment the counter

def convert_and_pickle(path='06-wildfire-sample'):
    image_files = find_image_files(path)
    mytensors = {}

    # Print each image file's full path
    for file in image_files:
        # print(file)
        mytensors[file] = onnx_pipeline.preprocess(file)
        print(f"Processed {file}")

    # Save the tensors as a pickle
    os.makedirs("pickled-files", exist_ok=True)
    with open("pickled-files/pickled_tensors.pkl", 'wb') as f:
        pickle.dump(mytensors, f)

# Call the function
convert_and_pickle()